In [1]:
import ee 
ee.Initialize()

EEException: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, and then retry.

In [4]:
#Pandas modules to interact with spatial data
import geopandas as gpd
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import Point,Polygon
from geemap import geopandas_to_ee

from pygeosys.util.dataframe import chunk_dataframe
#import numpy
import numpy as np
from functools import reduce

/home/ald/miniconda3/envs/pygeosys/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.0-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [5]:
### Parameters
year=2021
date_start = str(year)+'-01-01' # fall tillage : 08-01 to 11-01 ; spring tillage : 02-01 to 05-01
date_end = str(year)+'-12-01'
# input_filepath = '/home/ald/code/ukraine_monitoring/data/AOI/Hlobynska.gpkg'
# input_filepath = '/home/ald/code/ukraine_monitoring/data/AOI/Chudnivskyi.gpkg'
# input_filepath ='/home/ald/code/ukraine_monitoring/data/AOI/Lebedynska.gpkg'

input_filepath = '/home/ald/exemple.shp'
#Fetch features from earthengine geosys's assets
geometry_collection = gpd.read_file(input_filepath)

In [6]:
geometry_collection.head()

,id,geometry
0,None,"POLYGON ((1.57034 43.68988, 1.59634 43.68995, ..."


In [7]:
chunk_size=1
if chunk_size is None:
    geometry_collections = [geometry_collection]
else:
    geometry_collections = list(chunk_dataframe(geometry_collection, chunk_size))

print(f'Chunks: {len(geometry_collections)}')
filename_prefixes = []
for chunk_id,  gc in enumerate(geometry_collections):
    print(
            f'Processing chunk {chunk_id} / {len(geometry_collections)}')
    print(
            f'Uploading FeatureCollection ({len(gc)} Features) on server side')
    table = geopandas_to_ee(gc)


aoi = table.geometry()


Chunks: 1
Processing chunk 0 / 1
Uploading FeatureCollection (1 Features) on server side


In [8]:
aoi

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Collection.geometry",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection",
          "arguments": {
            "features": {
              "arrayValue": {
                "values": [
                  {
                    "functionInvocationValue": {
                      "functionName": "Feature",
                      "arguments": {
                        "geometry": {
                          "functionInvocationValue": {
                            "functionName": "GeometryConstructors.Polygon",
                            "arguments": {
                              "coordinates": {
                                "constantValue": [
                                  [
                                    [
                                      1.570337803949238,
                                      43.68988258799277
                                 

In [18]:
# prepare data before download
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD')
           .filterBounds(aoi)
           .filterDate(ee.Date(date_start),ee.Date(date_end))
           .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')))

In [19]:
# get available orbits info
im_coll.aggregate_array('relativeOrbitNumber_start').getInfo()


[110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110,
 110]

In [21]:
## get available dates 
import time
acq_times = im_coll.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

56

In [22]:
im_list = im_coll.toList(im_coll.size())

In [29]:
img=im_list.get(0)

In [44]:
img = ee.Image(im_coll.select('VV','VH').first())#get first image of collection 
img2 = ee.Image(im_coll.get(1))#get first image of collection 
imgMEAN = ee.Image(im_coll.filterDate(f'{year}-09-01', f'{year}-09-30').select('VV','VH').mean())

    

In [45]:
task=ee.batch.Export.image.toCloudStorage(**{
        'image': img.clip(aoi),
        'description': 'Gragnague SAR',
        'bucket': 'gri_geosys', #bucket name never changes
        'fileNamePrefix':'QLE/ChangeDetection/SAR/exempleGragnague',#path to file
        'scale': 10,#resolution
        'region':aoi,
        'maxPixels': 15000000000,
        'skipEmptyTiles': True
        })
   

In [46]:
task.start()

In [48]:
# Import the Folium library.
import folium
import sys  
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [57]:
location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(location=location, zoom_start=20)
mp.add_ee_layer(img,{'min': 0, 'max': 10},'Ratio')
mp.add_child(folium.LayerControl())

display(mp)
url = img.getThumbURL({'min': 0, 'max': 10})
disp.Image(url=url, width=800)

TypeError: 'int' object is not iterable